# Convolutional Neural Network Training

Author: Zhongyi (James) Guo <br>
Date: 11/07/2024

## Import Packages

In [1]:
# system
import os
import pickle
import gzip

# data manipulation
import pandas as pd
import numpy as np

# deep learning
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

2024-11-08 19:47:52.878423: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 19:47:52.895932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731095272.917475   97300 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731095272.923786   97300 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 19:47:52.945394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
os.getcwd()

'/home/ubuntu/SCA-DL-DGE/script/cnn'

## Import Dataset

In [3]:
with gzip.open('../../result/one_hot_encoding/gene_id_label_ohe.pkl.gz', 'rb') as f:
    data = pickle.load(f)

In [4]:
data

,ensembl_gene_id,DE,upstream_region_encoded
0,ENSG00000000457,1,"[[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
1,ENSG00000000460,0,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
2,ENSG00000000938,0,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [1,..."
3,ENSG00000000971,1,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
4,ENSG00000001460,0,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,..."
...,...,...,...
55221,ENSG00000284520,0,"[[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,..."
55222,ENSG00000284544,0,"[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1,..."
55223,ENSG00000284554,0,"[[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
55224,ENSG00000284568,0,"[[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0,..."


## Quality Control

In [5]:
X = np.stack(data['upstream_region_encoded'].values)
Y = data['DE'].values

In [6]:
print(X.shape)
print(Y.shape)

(55223, 2000, 4)
(55223,)


In [7]:
print(type(X))
print(type(Y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Train-test Split

In [8]:
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=123)

In [9]:
print(X_train.shape)
print(X_dev.shape)
print(Y_train.shape)
print(Y_dev.shape)

(44178, 2000, 4)
(11045, 2000, 4)
(44178,)
(11045,)


In [10]:
# add one more channel for the training purpose
X_train = np.expand_dims(X_train, axis=-1)
X_dev = np.expand_dims(X_dev, axis=-1) 
Y_train = np.expand_dims(Y_train, axis=-1) 
Y_dev = np.expand_dims(Y_dev, axis=-1) 

In [11]:
print(X_train.shape)
print(X_dev.shape)
print(Y_train.shape)
print(Y_dev.shape)

(44178, 2000, 4, 1)
(11045, 2000, 4, 1)
(44178, 1)
(11045, 1)


Hyperparameters:

In [12]:
BUFFER_SIZE = 50000
BATCH_SIZE = 256
EPOCH_SIZE = 50

Shuffle the dataset and apply mini-batch gradient descent with a batch size of 256:

In [13]:
train = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
dev = tf.data.Dataset.from_tensor_slices((X_dev, Y_dev))

I0000 00:00:1731095285.380319   97300 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13764 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5


In [14]:
for batch in train.take(1):  
    X_batch, Y_batch = batch
    print("X_batch:", X_batch.shape) 
    print("Y_batch:", Y_batch.shape)

X_batch: (256, 2000, 4, 1)
Y_batch: (256, 1)


2024-11-08 19:48:11.987789: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
num_batches = 0
for _ in train:
    num_batches += 1
print("Number of mini-batches:", num_batches)

Number of mini-batches: 173


2024-11-08 19:48:12.562740: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Model Building

In [16]:
model = models.Sequential()
model.add(layers.Input(shape=(2000, 4, 1), batch_size = BATCH_SIZE))
# Batch normalization (optional)
model.add(layers.Conv2D(16, (2, 2), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(32, (2, 2), activation='relu', padding='same'))
# Batch normalization (optional)
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (256, 2000, 4, 16)     │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (256, 1000, 2, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (256, 1000, 2, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (256, 500, 1, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (256, 32)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (256, 64)              │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (256, 10)              │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,922 (19.23 KB)

 Trainable params: 4,922 (19.23 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Model Training

In [19]:
history = model.fit(
    train,                  
    epochs=EPOCH_SIZE,        
    validation_data=dev     
)

Epoch 1/50


I0000 00:00:1731095294.411393   97378 service.cc:148] XLA service 0x70b8a8002fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731095294.411425   97378 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-11-08 19:48:14.440715: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731095294.569005   97378 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-08 19:48:15.481614: W external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:874] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2024-11-08 19:48:15.481655: W external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:877] Conv: %cudnn-conv.6 = (f32[16,32,2,2]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,256,1001,3]{3,2,1,0} %bitcast.1470, f32[32,256,1000,2]{3,2,1,0} %bitcas

  1/173 ━━━━━━━━━━━━━━━━━━━━ 13:21 5s/step - accuracy: 0.0000e+00 - loss: 2.3489

I0000 00:00:1731095297.911212   97378 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


172/173 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7424 - loss: 1.2638

2024-11-08 19:48:27.151718: W external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:874] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2024-11-08 19:48:27.151758: W external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:877] Conv: %cudnn-conv.6 = (f32[16,32,2,2]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,146,1001,3]{3,2,1,0} %bitcast.1470, f32[32,146,1000,2]{3,2,1,0} %bitcast.1472), window={size=1000x2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", metadata={op_type="Conv2DBackpropFilter" op_name="gradient_tape/sequential_1/conv2d_1_2/convolution/Conv2DBackpropFilter" source_file="/home/ubuntu/miniconda3/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1196}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_sc

173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7429 - loss: 1.2608

ValueError: Arguments `target` and `output` must have the same shape up until the last dimension: target.shape=(1,), output.shape=(2000, 10)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
# test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)